In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, sqrt, integrate

<h2>Task 1</h2>

<h3>Monte Carlo integration</h3>

The average value of a (continuous) function $\, f(x) \,$ can be written as

\begin{align*}
    \bar{f} &= \frac{1}{b - a} \int_{a}^{b} f(x) \, dx \\
    (b - a) \bar{f} &= \int_{a}^{b} f(x) \, dx \\
    (b - a) \frac{1}{n} \sum_{i=1}^{n} f(x_i) &\approx \int_{a}^{b} f(x) \, dx, 
\end{align*}

where $\, x_i, \, \, i=1,...,n, \,$ are random samples from $\, \text{Uniform}(a,b). \,$ This is the key idea behind Monte Carlo integration. This approximation works due to the law of large numbers; the sum $\, (b - a) \frac{1}{n} \sum_{i=1}^{n} f(x_i) \,$ is the sample mean of $\, f(x), \,$ and it converges to the true mean as $\, n \rightarrow \infty. \,$

**(a)**

In [2]:
def eval_integral(a, b):
    x = symbols('x')
    f = sqrt(1 + (1 / (1 + x)))
    integral = integrate(f, (x, a, b)).evalf()
    return integral

In [3]:
eval_integral(a=0, b=1)

1.30011842817113

- So the integral is approximately equal to 1.30012.
- Let's see how close we can get via Monte Carlo integration.

In [2]:
def f_X(x):
    return np.sqrt(1 + (1 / (1 + x)))

In [3]:
def monte_carlo_integration(n, a=0, b=1):
    """
    Args:
        a: lower limit of integration
        b: upper limit of integration
        n: the number of random samples to draw from U(a,b)
    """
    u = np.random.uniform(low=a, high=b, size=n)
    MC_est = np.sum(f_X(u))
    return (b-a) * (1/n) * MC_est

In [11]:
for n in [10, 100, 1000, 10000, 100000, 1000000]:
    mc_est = monte_carlo_integration(n=n)
    print(f'n: {n}, Monte Carlo estimator: {mc_est:.5f}')

n: 10, Monte Carlo estimator: 1.29837
n: 100, Monte Carlo estimator: 1.30252
n: 1000, Monte Carlo estimator: 1.29858
n: 10000, Monte Carlo estimator: 1.29964
n: 100000, Monte Carlo estimator: 1.29998
n: 1000000, Monte Carlo estimator: 1.30022


**(b)**

Let $\, \mathcal{\hat{I}} \,$ be the MC estimate of the integral, and let $\, \mathcal{I} \,$ be its true value. We want that (Chebysev's inequality)

\begin{equation*}
    P(|\mathcal{\hat{I}} - \mathcal{I}| < \epsilon) \ge 1 - \frac{\sigma^2}{\epsilon^2} \ge 0.99
\end{equation*}

Let $\, X_1,...,X_n \overset{\bot}{\sim}\text{Uniform}(0,1). \,$ The MC estimate and its variance are

\begin{equation*}
    \mathcal{\hat{I}} = \frac{1}{n} \sum_{i=1}^{n} f(X_i)
\end{equation*}

\begin{equation*}
    \text{Var}(\mathcal{\hat{I}}) = \text{Var} \left(\frac{1}{n} \sum_{i=1}^{n} f(X_i) \right) = \frac{1}{n^2} \sum_{i=1}^{n} \text{Var}(f(X_i)) = \frac{1}{n} \text{Var}(f(X))
\end{equation*}

We note that the theoretical variance $\, \sigma^2 \,$ can be replaced with the sample variance of the MC sample. Hence we have

\begin{align*}
    1 - \frac{\sigma^2}{\epsilon^2} &\ge 0.99 \\
    1 - \frac{\text{Var}(f(X))}{n \, \epsilon^2} &\ge 0.99 \\
    0.01 - \frac{\text{Var}(f(X))}{n \, \epsilon^2} &\ge 0 \\
    0.01 &\ge \frac{\text{Var}(f(X))}{n \, \epsilon^2} \\
    0.01 \cdot n \cdot \epsilon^2 &\ge \text{Var}(f(X)) \\
    n &\ge \frac{\text{Var}(f(X))}{0.01 \cdot \epsilon^2}.
\end{align*}

We are given that $\, \epsilon = 0.001, \,$ so we finally have the required sample size:

\begin{equation*}
    n \ge \frac{\text{Var}(f(X))}{0.01 \cdot 0.001^2}
\end{equation*}

In [12]:
def var_fX(n):
    u = np.random.uniform(low=0, high=1, size=n)
    fu = f_X(u)
    MC_estimator_var = np.var(fu, ddof=1)
    return MC_estimator_var

In [14]:
var_fX(n=1000000)

np.float64(0.002833616600076021)

In [15]:
def var_fX2(n):
    """
    Just confirming that the simplified version above gives the same result 
    as the formula provided in the lecture notes.
    """
    u = np.random.uniform(low=0, high=1, size=n)
    MC_est = np.sum(f_X(u)) * (1/n)
    return (1/(n-1)) * np.sum((f_X(u) - MC_est)**2)

In [16]:
var_fX2(n=1000)

np.float64(0.0028399843316707137)

In [17]:
def required_sample_size(s):
    sample_size = int(np.ceil(s / (0.01 * 0.001**2)))
    print(f'Required sample size: {sample_size}')
    return sample_size

In [18]:
sample_size = required_sample_size(s=var_fX(n=1000000))

Required sample size: 283753


<h2>Task 2</h2>

Let 

$$
p_{X}(x) =
\begin{cases} 
x & \text{, } 0 < x \le 1, \\
2 - x & \text{, } 1 < x \le 2, \\
0 & \text{, else}
\end{cases}
$$

Be the probability density function of a random variable $\, X. \,$ Our goal is to calculate the expectation

\begin{equation*}
    \mathbb{E}[f(X)] = \int_{-\infty}^{\infty} f(X) \, p_{X}(x) \, dx = \int_{0}^{2} f(X) \, p_{X}(x) \, dx
\end{equation*}

we'll do this via 1) *Monte Carlo integration* and 2) *importance sampling*. For the importance sampling, we are given that the proposal distribution is $\, X' \sim \text{Uniform}(0,2). \,$ We are also given that $\, f(X) = |1 - x|. \,$

In importance sampling, the expectation $\, \mathbb{E}[f(X)] \,$ can be approximated as

\begin{equation*}
    \mathbb{E}_p[f(X)] = \int_{\mathbb{X}} f(x) \, p_X(x) \, dx = \int_{\mathbb{X}} f(x) \, \frac{p_X(x)} {q_{X'}(x)} \, q_{X'}(x) \, dx = \mathbb{E}_q \left[f(X) \, \frac{p_X(X)}{q_{X'}(X)} \right] \approx \frac{1}{n} \sum_{i=1}^{n} f(X_i) \, \frac{p_X(X_i)}{q_{X'}(X_i)}.
\end{equation*}

$\large p_X(x)$ 
- the target density 
- the distribution that we're interested in
- we're computing $\, \mathbb{E}[f(X)] \,$ assuming that $\, X \sim p_X \,$

$\large q_{X'}(x)$ 
- the proposal density
- a simpler distribution that we're generating samples from
- we're using these samples to approximate $\, \mathbb{E}[f(X)], \,$ assuming that $\, X \sim p_X \,$

In [2]:
def f(x):
    return np.abs(1 - x)

In [3]:
def p(x):
    """
    Target density p_X(x)
    """
    return np.maximum(0, 1 - np.abs(1 - x))

In [4]:
def q(a, b):
    """
    Proposal density q_X'(x)
    """
    return 1 / (b - a)

In [5]:
def rejection_sampling(n):
    res = np.zeros(n)
    i = 0
    while i < n:
        x = np.random.uniform(low=0.0, high=2.0, size=1)
        y = np.random.uniform(low=0.0, high=2.0, size=1)
        if y <= p(x):
            res[i] = x.item()
            i += 1
    return res

In [6]:
def MC_integration(n=1000):
    X = rejection_sampling(n=n)
    MC_estimate = np.mean(f(X))
    print(f'E[f(X)] ≈ Q_n(f(X)) = {MC_estimate}')

In [7]:
def importance_sampling(a=0, b=2, n=1000):
    X = np.random.uniform(low=a, high=b, size=n)
    print(f'E[f(X)] ≈ {np.mean(f(X) * (p(X) / q(a,b)))}')

In [10]:
MC_integration()

E[f(X)] ≈ Q_n(f(X)) = 0.33446505010082955


In [9]:
importance_sampling()

E[f(X)] ≈ 0.33280822153461653


Next, we'll compare which of the methods yields a smaller variance.

In [15]:
def compare_vars(a=0, b=2, n=1000):
    # Monte Carlo
    u = rejection_sampling(n=n)
    fu = f(u)
    var1 = np.var(fu, ddof=1)
    
    # Importance sampling
    X = np.random.uniform(low=a, high=b, size=n)
    Y = f(X) * (p(X) / q(a,b))
    var2 = np.var(Y, ddof=1)
    
    return var1, var2

In [19]:
compare_vars()

(np.float64(0.055458503142710565), np.float64(0.02261144461553514))

- Importance sampling yields a smaller variance probably due to the fact that the proposal distribution $\, q_{X'}(x) \,$ is well chosen and resembles the target distribution $\, p_{X}(x). \,$

<h2>Task 4</h2>

We'll approximate the integral

$$ \int_{0}^{1} e^{-x^2} dx \approx 0.746824. $$

using Monte Carlo integration.

<h3>Regular Monte Carlo integration</h3>

\begin{align*}
    \int_{a}^{b} f(x) \, dx \approx (b-a) \frac{1}{n} \sum_{i=1}^{n} f(X_i).
\end{align*}

<h3>Antithetic variates method</h3>

First we draw $\, n/2 \,$ independent random variables from the uniform distribution: $\, X_1,...X_{n/2} \overset{\bot}{\sim} \text{Uniform}(0,1). \,$ Then we define a new random variable $\, Y_i = 1 - X_i. \,$ Our estimator is now:

\begin{equation*}
    \hat{\mu}_A = \frac{1}{n/2} \sum_{i=1}^{n/2} \frac{f(X_i) + f(Y_i)}{2} = \frac{1}{n} \sum_{i=1}^{n/2} (f(X_i) + f(Y_i)).
\end{equation*}

In [2]:
def f(x):
    return np.exp(-x**2)

In [5]:
def MC_int(n, a=0, b=1):
    x = np.random.uniform(low=a, high=b, size=n)
    return (b-a) * np.mean(f(x))

In [23]:
def antithetic_variates_method(n, a=0, b=1):
    X = np.random.uniform(low=a, high=b, size=n//2)
    Y = 1 - X
    return (1/n) * np.sum(f(X) + f(Y))

In [ ]:
def MC_sd(n, m):
    estimates = [MC_int(n=n) for _ in range(m)]
    return np.std(estimates, ddof=1)

In [28]:
def antithetic_sd(n, m):
    estimates = [antithetic_variates_method(n=n) for _ in range(m)]
    return np.std(estimates, ddof=1)

In [29]:
MC_int(n=1000000)

np.float64(0.7467665664410794)

In [30]:
antithetic_variates_method(n=1000000)

np.float64(0.7468596867693995)

In [37]:
MC_sd(n=100000, m=10000)

np.float64(0.0006384018199759097)

In [38]:
antithetic_sd(n=100000, m=10000)

np.float64(0.00012670032305264433)

- The antithetic variates method gives approximately 5 times smaller standard error (standard deviation).
- I believe that this is due to the fact that $\, X \,$ and $\, Y \,$ are negatively correlated.